In [1]:
import netCDF4 as nc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import geopandas
import xarray as xr
import matplotlib.colors as pltc
import geopandas
import datetime as dt
from scipy import stats
from sklearn import preprocessing
import s3fs
import matplotlib as mpl
import rasterio
from rasterio.crs import CRS
import rioxarray as rxr
import rasterstats as rs
mpl.rcParams['pdf.fonttype'] = 42

## Cubic feet to cubic meters conversion factor
cfs_2_cms = 0.0283168466

In [6]:
src = rasterio.open("../data/PROSPER/SPP_MEAN.tif")
srcArry = src.read(1)
srcArry[srcArry < 0] = np.nan ## Filter out 


pnwNP = pd.read_csv("../data/pnwNP_StatsLocations.csv")
pnwNP = geopandas.GeoDataFrame(pnwNP, geometry=geopandas.points_from_xy(pnwNP.dec_long_va,pnwNP.dec_lat_va)) \
.set_crs('epsg:4269',inplace=True)

pnwNP = pnwNP.to_crs(src.crs)
# pnwNP = pnwNP.iloc[2:4]
pnwNP['geometry'] = pnwNP.geometry.buffer(100)
pnwNP.to_file('pnwBufferTestDelete.shp')

<ipython-input-6-a3ea7623ac64>:13: UserWarning: Column names longer than 10 characters will be truncated when saved to ESRI Shapefile.
  pnwNP.to_file('pnwBufferTestDelete.shp')


In [7]:
tt = rs.zonal_stats("pnwBufferTestDelete.shp",
                    srcArry,
                    nodata=-999,
                    affine=src.transform,
                    geojson_out=True,
                    copy_properties=True,
                    stats="count min mean max median")

out = geopandas.GeoDataFrame.from_features(tt)

In [10]:
out[out['count'] == out['count'].max()]

,geometry,site_no,dec_lat_va,dec_long_v,gage,numZero,numOne,startDate,endDate,comid,numObs,propZero,propOne,min,max,mean,count,median
379,"POLYGON ((-1612470.974 2463724.994, -1612471.4...",13212000,43.616556,-116.217618,13212000,509,509,1966-04-02,1973-09-29,23398863.0,2737,0.18597,0.18597,0.182769,0.203538,0.191077,16,0.184154


In [11]:
out

,geometry,site_no,dec_lat_va,dec_long_v,gage,numZero,numOne,startDate,endDate,comid,numObs,propZero,propOne,min,max,mean,count,median
0,"POLYGON ((-1910972.850 2316027.184, -1910973.3...",10361700,41.722116,-119.373258,10361700,2797,3228,1963-10-02,1972-09-29,24061841.0,3285,0.851446,0.982648,0.112000,0.112769,0.112128,6,0.112000
1,"POLYGON ((-1948387.790 2365945.149, -1948388.2...",10366000,42.072111,-119.962725,10366000,0,61,1950-01-02,1991-10-17,24042897.0,15263,0.000000,0.003997,NaN,NaN,NaN,0,NaN
2,"POLYGON ((-1935422.497 2403343.606, -1935422.9...",10378500,42.424883,-119.923285,10378500,0,2405,1950-01-02,1991-09-30,24043081.0,15246,0.000000,0.157746,0.607846,0.607846,0.607846,3,0.607846
3,"POLYGON ((-1984354.041 2482580.910, -1984354.5...",10387500,42.999869,-120.751093,10387500,8177,8544,1951-06-19,1991-09-29,NaN,14712,0.555805,0.580750,0.074308,0.096923,0.085831,10,0.086308
4,"POLYGON ((-2006029.538 2501245.606, -2006030.0...",10389500,43.110973,-121.069165,10389500,3131,4183,1965-10-02,1991-09-29,24026288.0,9493,0.329822,0.440640,0.043231,0.590615,0.434198,7,0.590615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
613,"POLYGON ((-2248512.946 2444943.172, -2248513.4...",14375400,42.031501,-123.737848,14375400,0,10,1985-10-02,1991-09-30,23945187.0,2189,0.000000,0.004568,NaN,NaN,NaN,0,NaN
614,"POLYGON ((-1187484.294 2376505.808, -1187484.7...",433247110491701,43.546317,-110.822154,433247110491701,53,53,2008-07-23,2010-11-07,24433125.0,837,0.063321,0.063321,0.734923,0.737846,0.735569,5,0.735077
615,"POLYGON ((-2037592.723 2782272.259, -2037593.2...",452755122234100,45.465120,-122.395923,452755122234100,4,1007,2013-01-02,2019-10-14,23815490.0,2476,0.001616,0.406704,0.514308,0.618462,0.584365,8,0.602538
616,"POLYGON ((-2035081.886 2781966.662, -2035082.3...",452807122215001,45.468667,-122.363972,452807122215001,1,817,2015-10-29,2018-10-17,23815474.0,1084,0.000923,0.753690,0.383538,0.387385,0.385923,6,0.386154


In [16]:
out = out.drop(['geometry','gage'],axis=1)
out.columns = ['gage', 'dec_lat', 'dec_long', 'numZero',
       'numOne', 'startDate', 'endDate', 'comid', 'numObs', 'propZero',
       'propOne', 'minProb_SS', 'maxProb_SS', 'meanProb_SS', 'countProb_SS', 'medianProb_SS']

In [18]:
out.to_csv("../data/pnwNP_StatsLocations_SS.csv")

In [20]:
out[out['comid'].isna()]

,gage,dec_lat,dec_long,numZero,numOne,startDate,endDate,comid,numObs,propZero,propOne,minProb_SS,maxProb_SS,meanProb_SS,countProb_SS,medianProb_SS
3,10387500,42.999869,-120.751093,8177,8544,1951-06-19,1991-09-29,NaN,14712,0.555805,0.580750,0.074308,0.096923,0.085831,10,0.086308
70,12118300,47.362323,-121.997059,0,186,1956-09-02,1958-09-29,NaN,757,0.000000,0.245707,0.420000,0.423231,0.421282,6,0.420538
71,12119730,47.600100,-122.142900,277,738,1979-12-27,1982-01-31,NaN,766,0.361619,0.963446,NaN,NaN,NaN,0,NaN
104,12202400,48.748721,-122.409333,1311,5958,2001-10-02,2020-12-31,NaN,7030,0.186486,0.847511,0.483385,0.485846,0.484462,6,0.484231
105,12202450,48.769277,-122.406555,178,4560,2001-10-02,2020-12-31,NaN,7030,0.025320,0.648649,0.435846,0.436154,0.436000,4,0.436000
162,12447285,48.283205,-119.708394,0,236,2012-10-01,2020-12-31,NaN,3013,0.000000,0.078327,0.323846,0.511846,0.488269,8,0.511846
288,13132515,43.495556,-113.073333,8467,8503,1984-06-19,2009-09-29,NaN,9233,0.917037,0.920936,NaN,NaN,NaN,0,NaN
293,1313457010,42.829624,-114.937837,348,364,1988-04-12,1997-09-29,NaN,3457,0.100665,0.105294,NaN,NaN,NaN,0,NaN
316,13155750,42.936944,-115.382778,557,628,1988-05-01,1995-11-30,NaN,2769,0.201156,0.226797,NaN,NaN,NaN,0,NaN
317,13155800,42.936667,-115.382778,732,751,1985-04-06,2001-09-29,NaN,6020,0.121595,0.124751,NaN,NaN,NaN,0,NaN
